In [1]:
import tkinter as tk
import numpy as np
import joblib
import cv2
from PIL import ImageTk, Image
from tkinter import filedialog
import pandas as pd
import random
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings("ignore")
from scipy.fftpack import dct
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import MinMaxScaler
from tkinter import Tk, Text, font

In [2]:
def find_start_points_of_blocks(size, split_size, overlap=0):
    points = [0]
    stride = int(split_size * (1-overlap))
    counter = 1
    while True:
        pt = stride * counter
        if pt + split_size >= size:
            points.append(size - split_size)
            break
        else:
            points.append(pt)
        counter += 1
    return points

In [3]:
def extract_features(path_to_img):
    
    dataframe_list = []

    img = cv2.imread(path_to_img)

    # if the image file is empty or didn't load continue from beginining
    if np.shape(img) == ():
        return

    # converts the image into YCrCb color space and take the Cr component only
    img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb) 
    img = img[:,:,1]
    img_h, img_w = img.shape


    # calculates the starting points of the blocks
    X_points = find_start_points_of_blocks(img_w, 16, 0.0)
    Y_points = find_start_points_of_blocks(img_h, 16, 0.0)


    # calculates dct for eact block of an image
    dct_blocks=[]
    for i in Y_points:
        for j in X_points:
            block = img[i:i+16, j:j+16] # contains the non-overlapping block 
            lbp = local_binary_pattern(block, 8, 1.0, method='default')
            dct_block = dct(lbp, type=2, n=None, axis=-1, norm=None, overwrite_x=False)
            dct_blocks.append(dct_block)

    dct_blocks_array=np.asarray(dct_blocks)

    _,r,c=dct_blocks_array.shape


    img_std_list=[] #length should be r*c i.e 16*16=256 in our case.
    with_name_list=[path_to_img,1]

    for x in range(r):
        for y in range(c):
            pixel_depth_subarr = dct_blocks_array[:,x,y]
            std = np.std(pixel_depth_subarr)
            img_std_list.append(std)
            with_name_list.append(std)

    dataframe_list.append(with_name_list)
    
    df = pd.DataFrame(dataframe_list)
    df.rename(columns = {0: "image_names"}, inplace = True)
    
    array=df.values
    x_feature=array[:,2:]
    return x_feature

In [4]:
def open_n_check_image():
    text_widget.delete(1.0, tk.END)
    text_widget.configure(bg="white")
    # Open a file dialog to select an image file
    file_path = filedialog.askopenfilename(title="Select Image File", filetypes=(
    ("Image files", "*.jpg;*.jpeg;*.png;*.bmp"), ("All files", "*.*")))

    # Check if a file was selected
    if file_path:
        # Open the selected image using PIL
        image = Image.open(file_path)

        # Resize the image to fit within the window
        image = image.resize((400, 400), Image.ANTIALIAS)

        # Create an ImageTk object to display the image in the GUI
        image_tk = ImageTk.PhotoImage(image)

        # Update the image label with the new image
        image_label.configure(image=image_tk)
        image_label.image = image_tk
        
        # Load the saved model
        model = joblib.load('E:\Image Forgery Detection\codes\model_trained_on_casia2_dataset.joblib')
        
        # extract feature
        features = extract_features(file_path)
        
        
        # predict class
        predicted_class=model.predict(features)
        print(predicted_class)
        text = "ORIGINAL"
        if(predicted_class==1):
            text = "TAMPERED"
            text_widget.configure(bg="red")
        else:
            text_widget.configure(bg="green")
        
        text_widget.insert(tk.END, text)

In [ ]:
# Create the main window
window = tk.Tk()
window.title("Image Forgery Checker") 
window.geometry("800x600")

# Create a button to open the image
open_button = tk.Button(window, text="Open and Check Image", command=open_n_check_image)
open_button.pack(pady=10)

# Create a label to display the image
image_label = tk.Label(window)
image_label.pack()

# Create a custom font
custom_font = font.Font(size=13)

# Create a Text widget
text_widget = tk.Text(window, font = custom_font,  width=10, height=1)
text_widget.pack(pady=20)

# Start the GUI event loop
window.mainloop()

[0]
